In [13]:
%run moon_code
%run xgb_modelfit

In [14]:
import warnings
warnings.filterwarnings(action='ignore')

# 문교수님 지시사항 이행

>### 1. Check XGB and Feature Selection
>### 2. Calculate Portfolio Allocation
>### 3. 다중 Class

# 1. Check XGB and Feature Selection

In [18]:
# Data load
with open('raw_data_20181008.pickle', 'rb') as handle:
    raw_data = pickle.load(handle)
# 사용할 12개 주식 목록 
stock_li = ['현대차', 'SK텔레콤', 'LG화학', '셀트리온', 
            'KB금융', '삼성전자', 'SK하이닉스', 'NAVER', 
            '현대글로비스', '현대건설', '삼성중공업', 'POSCO']
# 12개 주식만 남기기
key_li = list(raw_data.keys())
for i in key_li:
    if i not in stock_li:
        del raw_data[i]
# KOSPI data load
with open('ks11_20181008.pickle', 'rb') as handle:
    ks11 = pickle.load(handle)
# KOSPI 수익률 추가
for i in [1,3,5,7,10]:
    ks11['returnsClosePrevRaw{}'.format(i)] = np.log(ks11.Open / ks11.Close.shift(i))
    ks11['returnsOpenPrevRaw{}'.format(i)] = np.log(ks11.Open / ks11.Open.shift(i))
# KOSPI Volume==0인 index 저장
kospi_index_del = ks11[ks11.Volume == 0][ks11.columns[:6]].index
# KOSPI Volume ==0인 index, stock에서 제외
for key, df in raw_data.items():
    raw_data[key] = df.drop(ks11[ks11.Volume == 0][ks11.columns[:6]].index, axis=0)
# stock에서 Volume==0인 index, stock에서 제외
for key, df in raw_data.items():
    raw_data[key] = df.drop(df.index[np.where(df.Volume==0)[0]], axis=0)
# feature 생성
for key, df in raw_data.items():
    df["pct_change"] = df.Close.pct_change()
    for i in range(3, 31, 2):
        df["close_ma_{}".format(i)] = df.Close.rolling(window = i).mean()
        df["volume_ma_{}".format(i)] = df.Volume.rolling(window = i).mean()
    for i in [7, 21, 30, 60, 90, 91, 100, 120]:
        df["his_vol_{}".format(i)] = df["pct_change"].rolling(window = i).std()*(252**0.5)
    for i in [1, 3, 5, 7, 10]:
        df['returnsClosePrevRaw{}'.format(i) ]= np.log(df.Open / df.Close.shift(i))
        df['returnsOpenPrevRaw{}'.format(i) ] = np.log(df.Open / df.Open.shift(i))
        df['returnsClosePrevMktres{}'.format(i)] = df['returnsClosePrevRaw{}'.format(i)] - ks11['returnsClosePrevRaw{}'.format(i)]
        df['returnsOpenPrevMktres{}'.format(i)] = df['returnsOpenPrevRaw{}'.format(i)] - ks11['returnsOpenPrevRaw{}'.format(i)]
    for i in [1, 3, 5, 7, 10]:
        df['Y3_{}'.format(i)] = np.where(df.Close.rolling(i).mean() - df.Close.shift(1).rolling(i).mean()>=0,1,0)
        df['sh1_Y3_{}'.format(i)] = df['Y3_{}'.format(i)].shift(-1)
        df['sh5_Y3_{}'.format(i)] = df['Y3_{}'.format(i)].shift(-5)
# 날짜 조정
data = adjust_window(raw_data, '2013-07-01', '2018-07-01')

li = ['Y3_{}'.format(i) for i in [1,3,5,7,10]]
li.extend(['sh1_Y3_{}'.format(i) for i in [1,3,5,7,10]])
li.extend(['sh5_Y3_{}'.format(i) for i in [1,3,5,7,10]])
target_li = li

X_li = [i for i in data['삼성전자'].columns if i not in target_li]
scl_X_li = ['scl_'+i for i in X_li]

for key, df in data.items():
    for X in X_li:
        df['scl_{}'.format(X)] = df[X]
        
train, test = multi_train_test_split(data, 0.8, dtrain=True)

multi_apply_standard_scale(train, test, scl_X_li)

scl_li = scl_X_li.copy()
for ix, i in enumerate(scl_X_li):
    if (i.find('returns') != -1) or (i.find('his') != -1) or (i.find('pct') != -1):
        scl_li[ix] = scl_X_li[ix][4:]
origin_X = [i for i in X_li if i.find('returns') != 0]
origin_scl_X = [i for i in scl_li if i.find('returns') != 0]

In [217]:
store = (stock_li, ks11, raw_data, train, test)
li = (X_li, scl_li, origin_X, origin_scl_X, target_li)
with open('store_data_20181013.pickle', 'wb') as handle:
    pickle.dump(store, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('store_data_2_20181013.pickle', 'wb') as handle:
    pickle.dump(li, handle, protocol=pickle.HIGHEST_PROTOCOL)

> ## 실험 1-1
- predictors : X_li
- target : sh1_Y3_1

In [137]:
predictors = X_li.copy()
target = 'sh1_Y3_1'
result, fi_dict = temp_calc_feature_importance(alg, train, test, predictors, target)

In [138]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc'])

,KB금융,LG화학,NAVER,POSCO,SK텔레콤,SK하이닉스,삼성전자,삼성중공업,셀트리온,현대건설,현대글로비스,현대차
acc,0.471074,0.425620,0.537815,0.533058,0.504132,0.512397,0.487603,0.537190,0.512397,0.479339,0.561983,0.537190
f1,0.536232,0.125786,0.537815,0.638978,0.491525,0.427184,0.569444,0.517241,0.448598,0.479339,0.598485,0.608392
pre,0.592000,0.075758,0.571429,0.740741,0.504348,0.335878,0.656000,0.504202,0.380952,0.500000,0.658333,0.731092
rec,0.490066,0.370370,0.507937,0.561798,0.479339,0.586667,0.503067,0.530973,0.545455,0.460317,0.548611,0.520958
auc,0.464813,0.401464,0.539683,0.507461,0.504132,0.532854,0.479382,0.536805,0.519481,0.480159,0.565122,0.547146


In [139]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc']).mean(1)

acc    0.508317
f1     0.498252
pre    0.520894
rec    0.508797
auc    0.506542
dtype: float64

In [140]:
pd.DataFrame(pd.DataFrame(fi_dict).count(1).sort_values(ascending=False)[:10])

,0
pct_change,9
Volume,8
returnsClosePrevMktres1,7
volume_ma_3,7
returnsClosePrevMktres3,7
returnsClosePrevRaw1,6
returnsClosePrevMktres7,6
returnsClosePrevRaw3,5
his_vol_100,4
returnsOpenPrevMktres3,4


> ## 실험 1-2
- predictors : scl_li
- target : sh1_Y3_1

In [141]:
predictors = scl_li.copy()
target = 'sh1_Y3_1'
result, fi_dict = temp_calc_feature_importance(alg, train, test, predictors, target)

In [142]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc'])

,KB금융,LG화학,NAVER,POSCO,SK텔레콤,SK하이닉스,삼성전자,삼성중공업,셀트리온,현대건설,현대글로비스,현대차
acc,0.471074,0.425620,0.537815,0.533058,0.504132,0.512397,0.487603,0.537190,0.512397,0.479339,0.561983,0.533058
f1,0.536232,0.125786,0.537815,0.638978,0.491525,0.427184,0.569444,0.517241,0.448598,0.479339,0.598485,0.606272
pre,0.592000,0.075758,0.571429,0.740741,0.504348,0.335878,0.656000,0.504202,0.380952,0.500000,0.658333,0.731092
rec,0.490066,0.370370,0.507937,0.561798,0.479339,0.586667,0.503067,0.530973,0.545455,0.460317,0.548611,0.517857
auc,0.464813,0.401464,0.539683,0.507461,0.504132,0.532854,0.479382,0.536805,0.519481,0.480159,0.565122,0.542712


In [143]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc']).mean(1)

acc    0.507972
f1     0.498075
pre    0.520894
rec    0.508538
auc    0.506172
dtype: float64

In [144]:
pd.DataFrame(pd.DataFrame(fi_dict).count(1).sort_values(ascending=False)[:10])

,0
pct_change,9
scl_Volume,8
returnsClosePrevMktres1,7
scl_volume_ma_3,7
returnsClosePrevMktres3,7
returnsClosePrevRaw1,6
returnsClosePrevMktres7,6
returnsClosePrevRaw3,5
returnsOpenPrevMktres7,4
returnsOpenPrevMktres3,4


> ## 실험 1-3
- predictors : X_li
- target : sh5_Y3_1

In [145]:
predictors = X_li.copy()
target = 'sh5_Y3_1'
result, fi_dict = temp_calc_feature_importance(alg, train, test, predictors, target)

In [146]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc'])

,KB금융,LG화학,NAVER,POSCO,SK텔레콤,SK하이닉스,삼성전자,삼성중공업,셀트리온,현대건설,현대글로비스,현대차
acc,0.537190,0.471074,0.449580,0.516529,0.537190,0.483471,0.524793,0.520661,0.508264,0.483471,0.483471,0.528926
f1,0.636364,0.238095,0.543554,0.440191,0.461538,0.564460,0.622951,0.567164,0.522088,0.458874,0.531835,0.629870
pre,0.796748,0.151515,0.690265,0.345865,0.424779,0.618321,0.766129,0.638655,0.515873,0.456897,0.581967,0.808333
rec,0.529730,0.555556,0.448276,0.605263,0.505263,0.519231,0.524862,0.510067,0.528455,0.460870,0.489655,0.515957
auc,0.545567,0.505933,0.450700,0.540583,0.531543,0.468918,0.524726,0.523851,0.507925,0.482403,0.481941,0.545016


In [147]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc']).mean(1)

acc    0.503718
f1     0.518082
pre    0.566279
rec    0.516099
auc    0.509092
dtype: float64

In [148]:
pd.DataFrame(pd.DataFrame(fi_dict).count(1).sort_values(ascending=False)[:10])

,0
returnsClosePrevRaw1,8
Volume,7
returnsClosePrevMktres1,7
pct_change,6
returnsOpenPrevRaw10,5
returnsOpenPrevMktres3,5
volume_ma_5,5
returnsClosePrevMktres3,4
returnsOpenPrevRaw5,4
his_vol_7,4


> ## 실험 1-4
- predictors : scl_li
- target : sh5_Y3_1

In [149]:
predictors = scl_li.copy()
target = 'sh5_Y3_1'
result, fi_dict = temp_calc_feature_importance(alg, train, test, predictors, target)

In [150]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc'])

,KB금융,LG화학,NAVER,POSCO,SK텔레콤,SK하이닉스,삼성전자,삼성중공업,셀트리온,현대건설,현대글로비스,현대차
acc,0.537190,0.471074,0.449580,0.516529,0.537190,0.483471,0.524793,0.520661,0.508264,0.483471,0.483471,0.528926
f1,0.636364,0.238095,0.543554,0.440191,0.461538,0.564460,0.622951,0.567164,0.522088,0.458874,0.531835,0.629870
pre,0.796748,0.151515,0.690265,0.345865,0.424779,0.618321,0.766129,0.638655,0.515873,0.456897,0.581967,0.808333
rec,0.529730,0.555556,0.448276,0.605263,0.505263,0.519231,0.524862,0.510067,0.528455,0.460870,0.489655,0.515957
auc,0.545567,0.505933,0.450700,0.540583,0.531543,0.468918,0.524726,0.523851,0.507925,0.482403,0.481941,0.545016


In [151]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc']).mean(1)

acc    0.503718
f1     0.518082
pre    0.566279
rec    0.516099
auc    0.509092
dtype: float64

In [152]:
pd.DataFrame(pd.DataFrame(fi_dict).count(1).sort_values(ascending=False)[:10])

,0
returnsClosePrevRaw1,8
scl_Volume,7
returnsClosePrevMktres1,7
pct_change,6
scl_volume_ma_5,5
returnsOpenPrevMktres3,5
returnsOpenPrevRaw10,5
returnsOpenPrevRaw1,4
returnsClosePrevRaw3,4
returnsOpenPrevMktres10,4


> ## 실험 2-1
- predictors : X_li
- target : sh1_Y3_3

In [153]:
predictors = X_li.copy()
target = 'sh1_Y3_3'
result, fi_dict = temp_calc_feature_importance(alg, train, test, predictors, target)

In [154]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc'])

,KB금융,LG화학,NAVER,POSCO,SK텔레콤,SK하이닉스,삼성전자,삼성중공업,셀트리온,현대건설,현대글로비스,현대차
acc,0.694215,0.723140,0.747899,0.690083,0.677686,0.719008,0.714876,0.677686,0.723140,0.710744,0.694215,0.714876
f1,0.666667,0.733068,0.729730,0.708171,0.669492,0.736434,0.727273,0.576087,0.737255,0.708333,0.663636,0.687783
pre,0.601626,0.754098,0.743119,0.674074,0.675214,0.730769,0.773109,0.490741,0.691176,0.745614,0.669725,0.678571
rec,0.747475,0.713178,0.716814,0.745902,0.663866,0.742188,0.686567,0.697368,0.789916,0.674603,0.657658,0.697248
auc,0.702409,0.723846,0.746407,0.689617,0.677461,0.717585,0.718284,0.683022,0.724226,0.712302,0.691424,0.713286


In [155]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc']).mean(1)

acc    0.707297
f1     0.695327
pre    0.685653
rec    0.711065
auc    0.708322
dtype: float64

In [156]:
pd.DataFrame(pd.DataFrame(fi_dict).count(1).sort_values(ascending=False)[:10])

,0
pct_change,12
returnsClosePrevRaw3,12
returnsOpenPrevRaw1,11
returnsClosePrevRaw1,7
Volume,7
returnsClosePrevMktres1,6
returnsOpenPrevMktres10,6
his_vol_7,6
volume_ma_5,4
returnsClosePrevMktres3,4


> ## 실험 2-2
- predictors : scl_li
- target : sh1_Y3_3

In [157]:
predictors = scl_li.copy()
target = 'sh1_Y3_3'
result, fi_dict = temp_calc_feature_importance(alg, train, test, predictors, target)

In [158]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc'])

,KB금융,LG화학,NAVER,POSCO,SK텔레콤,SK하이닉스,삼성전자,삼성중공업,셀트리온,현대건설,현대글로비스,현대차
acc,0.694215,0.723140,0.747899,0.690083,0.677686,0.719008,0.714876,0.677686,0.723140,0.710744,0.694215,0.714876
f1,0.666667,0.733068,0.729730,0.708171,0.669492,0.736434,0.727273,0.576087,0.737255,0.708333,0.663636,0.687783
pre,0.601626,0.754098,0.743119,0.674074,0.675214,0.730769,0.773109,0.490741,0.691176,0.745614,0.669725,0.678571
rec,0.747475,0.713178,0.716814,0.745902,0.663866,0.742188,0.686567,0.697368,0.789916,0.674603,0.657658,0.697248
auc,0.702409,0.723846,0.746407,0.689617,0.677461,0.717585,0.718284,0.683022,0.724226,0.712302,0.691424,0.713286


In [159]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc']).mean(1)

acc    0.707297
f1     0.695327
pre    0.685653
rec    0.711065
auc    0.708322
dtype: float64

In [160]:
pd.DataFrame(pd.DataFrame(fi_dict).count(1).sort_values(ascending=False)[:10])

,0
returnsClosePrevRaw3,12
pct_change,12
returnsOpenPrevRaw1,11
returnsClosePrevRaw1,7
scl_Volume,7
returnsOpenPrevMktres10,6
returnsClosePrevMktres1,6
his_vol_7,6
returnsClosePrevMktres3,4
scl_volume_ma_5,4


> ## 실험 2-3
- predictors : X_li
- target : sh5_Y3_3

In [161]:
predictors = X_li.copy()
target = 'sh5_Y3_3'
result, fi_dict = temp_calc_feature_importance(alg, train, test, predictors, target)

In [162]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc'])

,KB금융,LG화학,NAVER,POSCO,SK텔레콤,SK하이닉스,삼성전자,삼성중공업,셀트리온,현대건설,현대글로비스,현대차
acc,0.545455,0.475207,0.529412,0.446281,0.553719,0.578512,0.524793,0.524793,0.504132,0.516529,0.479339,0.471074
f1,0.556452,0.169935,0.559055,0.385321,0.608696,0.627737,0.584838,0.206897,0.268293,0.493506,0.491935,0.319149
pre,0.570248,0.106557,0.628319,0.323077,0.750000,0.666667,0.692308,0.138889,0.164179,0.500000,0.564815,0.265487
rec,0.543307,0.419355,0.503546,0.477273,0.512195,0.593103,0.506250,0.405405,0.733333,0.487179,0.435714,0.400000
auc,0.545567,0.451384,0.535278,0.452922,0.576610,0.574902,0.533613,0.475873,0.602516,0.515590,0.487465,0.451497


In [163]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc']).mean(1)

acc    0.512437
f1     0.439318
pre    0.447545
rec    0.501388
auc    0.516935
dtype: float64

In [164]:
pd.DataFrame(pd.DataFrame(fi_dict).count(1).sort_values(ascending=False)[:10])

,0
his_vol_100,6
volume_ma_3,5
pct_change,5
his_vol_30,5
returnsOpenPrevRaw5,5
returnsOpenPrevRaw10,5
close_ma_11,4
volume_ma_7,4
his_vol_21,4
Volume,4


> ## 실험 2-4
- predictors : scl_li
- target : sh5_Y3_3

In [165]:
predictors = scl_li.copy()
target = 'sh5_Y3_3'
result, fi_dict = temp_calc_feature_importance(alg, train, test, predictors, target)

In [166]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc'])

,KB금융,LG화학,NAVER,POSCO,SK텔레콤,SK하이닉스,삼성전자,삼성중공업,셀트리온,현대건설,현대글로비스,현대차
acc,0.545455,0.475207,0.529412,0.446281,0.553719,0.578512,0.524793,0.524793,0.504132,0.516529,0.479339,0.471074
f1,0.556452,0.169935,0.559055,0.385321,0.608696,0.627737,0.584838,0.206897,0.268293,0.493506,0.491935,0.319149
pre,0.570248,0.106557,0.628319,0.323077,0.750000,0.666667,0.692308,0.138889,0.164179,0.500000,0.564815,0.265487
rec,0.543307,0.419355,0.503546,0.477273,0.512195,0.593103,0.506250,0.405405,0.733333,0.487179,0.435714,0.400000
auc,0.545567,0.451384,0.535278,0.452922,0.576610,0.574902,0.533613,0.475873,0.602516,0.515590,0.487465,0.451497


In [167]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc']).mean(1)

acc    0.512437
f1     0.439318
pre    0.447545
rec    0.501388
auc    0.516935
dtype: float64

In [168]:
pd.DataFrame(pd.DataFrame(fi_dict).count(1).sort_values(ascending=False)[:10])

,0
his_vol_100,6
his_vol_30,5
returnsOpenPrevRaw5,5
pct_change,5
returnsOpenPrevRaw10,5
scl_volume_ma_3,5
his_vol_21,4
scl_volume_ma_7,4
returnsClosePrevMktres1,4
scl_Volume,4


> ## 실험 3-1
- predictors : X_li
- target : sh1_Y3_5

In [169]:
predictors = X_li.copy()
target = 'sh1_Y3_5'
result, fi_dict = temp_calc_feature_importance(alg, train, test, predictors, target)

In [170]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc'])

,KB금융,LG화학,NAVER,POSCO,SK텔레콤,SK하이닉스,삼성전자,삼성중공업,셀트리온,현대건설,현대글로비스,현대차
acc,0.780992,0.814050,0.815126,0.805785,0.743802,0.801653,0.797521,0.801653,0.814050,0.776860,0.785124,0.785124
f1,0.755760,0.832714,0.777778,0.809717,0.725664,0.833333,0.796680,0.766990,0.835165,0.778689,0.752381,0.747573
pre,0.719298,0.842105,0.785714,0.775194,0.738739,0.863309,0.842105,0.745283,0.791667,0.753968,0.718182,0.747573
rec,0.796117,0.823529,0.770000,0.847458,0.713043,0.805369,0.755906,0.790000,0.883721,0.805085,0.790000,0.747573
auc,0.782950,0.812708,0.808913,0.806793,0.742349,0.800534,0.799692,0.799930,0.809117,0.777542,0.785845,0.780261


In [171]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc']).mean(1)

acc    0.793478
f1     0.784370
pre    0.776928
rec    0.793983
auc    0.792220
dtype: float64

In [172]:
pd.DataFrame(pd.DataFrame(fi_dict).count(1).sort_values(ascending=False)[:10])

,0
pct_change,12
returnsClosePrevRaw5,12
returnsOpenPrevRaw3,11
returnsOpenPrevMktres3,10
returnsClosePrevMktres1,8
returnsClosePrevRaw3,8
returnsOpenPrevRaw5,6
returnsOpenPrevRaw10,4
returnsOpenPrevMktres10,4
volume_ma_3,4


> ## 실험 3-2
- predictors : scl_li
- target : sh1_Y3_5

In [173]:
predictors = scl_li.copy()
target = 'sh1_Y3_5'
result, fi_dict = temp_calc_feature_importance(alg, train, test, predictors, target)

In [174]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc'])

,KB금융,LG화학,NAVER,POSCO,SK텔레콤,SK하이닉스,삼성전자,삼성중공업,셀트리온,현대건설,현대글로비스,현대차
acc,0.780992,0.814050,0.815126,0.805785,0.743802,0.801653,0.797521,0.801653,0.814050,0.776860,0.785124,0.785124
f1,0.755760,0.832714,0.777778,0.809717,0.725664,0.833333,0.796680,0.766990,0.835165,0.778689,0.752381,0.747573
pre,0.719298,0.842105,0.785714,0.775194,0.738739,0.863309,0.842105,0.745283,0.791667,0.753968,0.718182,0.747573
rec,0.796117,0.823529,0.770000,0.847458,0.713043,0.805369,0.755906,0.790000,0.883721,0.805085,0.790000,0.747573
auc,0.782950,0.812708,0.808913,0.806793,0.742349,0.800534,0.799692,0.799930,0.809117,0.777542,0.785845,0.780261


In [175]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc']).mean(1)

acc    0.793478
f1     0.784370
pre    0.776928
rec    0.793983
auc    0.792220
dtype: float64

In [176]:
pd.DataFrame(pd.DataFrame(fi_dict).count(1).sort_values(ascending=False)[:10])

,0
returnsClosePrevRaw5,12
pct_change,12
returnsOpenPrevRaw3,11
returnsOpenPrevMktres3,10
returnsClosePrevRaw3,8
returnsClosePrevMktres1,8
returnsOpenPrevRaw5,6
returnsOpenPrevRaw10,4
returnsClosePrevRaw1,4
returnsClosePrevMktres3,4


> ## 실험 3-3
- predictors : X_li
- target : sh5_Y3_5

In [177]:
predictors = X_li.copy()
target = 'sh5_Y3_5'
result, fi_dict = temp_calc_feature_importance(alg, train, test, predictors, target)

In [178]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc'])

,KB금융,LG화학,NAVER,POSCO,SK텔레콤,SK하이닉스,삼성전자,삼성중공업,셀트리온,현대건설,현대글로비스,현대차
acc,0.541322,0.466942,0.470588,0.487603,0.623967,0.524793,0.495868,0.553719,0.429752,0.516529,0.438017,0.475207
f1,0.584270,0.085106,0.496000,0.031250,0.564593,0.584838,0.537879,0.205882,0.148148,0.470588,0.480916,0.136054
pre,0.690265,0.046512,0.613861,0.015873,0.546296,0.591241,0.645455,0.133333,0.085106,0.419355,0.583333,0.097087
rec,0.506494,0.500000,0.416107,1.000000,0.584158,0.578571,0.461039,0.451613,0.571429,0.536082,0.409091,0.227273
auc,0.554383,0.482609,0.488953,0.741667,0.618320,0.514776,0.508929,0.510167,0.493859,0.519765,0.448864,0.378788


In [179]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc']).mean(1)

acc    0.502026
f1     0.360460
pre    0.372310
rec    0.520155
auc    0.521757
dtype: float64

In [180]:
pd.DataFrame(pd.DataFrame(fi_dict).count(1).sort_values(ascending=False)[:10])

,0
volume_ma_9,8
his_vol_120,6
volume_ma_7,5
his_vol_100,5
his_vol_21,5
his_vol_60,5
Volume,4
his_vol_30,4
his_vol_91,4
volume_ma_27,4


> ## 실험 3-4
- predictors : scl_li
- target : sh5_Y3_5

In [181]:
predictors = scl_li.copy()
target = 'sh5_Y3_5'
result, fi_dict = temp_calc_feature_importance(alg, train, test, predictors, target)

In [182]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc'])

,KB금융,LG화학,NAVER,POSCO,SK텔레콤,SK하이닉스,삼성전자,삼성중공업,셀트리온,현대건설,현대글로비스,현대차
acc,0.541322,0.466942,0.470588,0.487603,0.623967,0.524793,0.495868,0.553719,0.429752,0.516529,0.438017,0.475207
f1,0.584270,0.085106,0.496000,0.031250,0.564593,0.584838,0.537879,0.205882,0.148148,0.470588,0.480916,0.136054
pre,0.690265,0.046512,0.613861,0.015873,0.546296,0.591241,0.645455,0.133333,0.085106,0.419355,0.583333,0.097087
rec,0.506494,0.500000,0.416107,1.000000,0.584158,0.578571,0.461039,0.451613,0.571429,0.536082,0.409091,0.227273
auc,0.554383,0.482609,0.488953,0.741667,0.618320,0.514776,0.508929,0.510167,0.493859,0.519765,0.448864,0.378788


In [183]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc']).mean(1)

acc    0.502026
f1     0.360460
pre    0.372310
rec    0.520155
auc    0.521757
dtype: float64

In [184]:
pd.DataFrame(pd.DataFrame(fi_dict).count(1).sort_values(ascending=False)[:10])

,0
scl_volume_ma_9,8
his_vol_120,6
scl_volume_ma_7,5
his_vol_21,5
his_vol_60,5
his_vol_100,5
scl_Volume,4
his_vol_30,4
his_vol_91,4
scl_volume_ma_27,4


> ## 실험 4-1
- predictors : X_li
- target : sh1_Y3_7

In [185]:
predictors = X_li.copy()
target = 'sh1_Y3_7'
result, fi_dict = temp_calc_feature_importance(alg, train, test, predictors, target)

In [186]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc'])

,KB금융,LG화학,NAVER,POSCO,SK텔레콤,SK하이닉스,삼성전자,삼성중공업,셀트리온,현대건설,현대글로비스,현대차
acc,0.826446,0.822314,0.848739,0.838843,0.838843,0.809917,0.822314,0.847107,0.776860,0.818182,0.863636,0.809917
f1,0.820513,0.846975,0.820000,0.844622,0.828194,0.836879,0.821577,0.827907,0.800000,0.828125,0.845070,0.778846
pre,0.800000,0.862319,0.796117,0.828125,0.839286,0.848921,0.818182,0.831776,0.794118,0.803030,0.833333,0.750000
rec,0.842105,0.832168,0.845361,0.861789,0.817391,0.825175,0.825000,0.824074,0.805970,0.854839,0.857143,0.810000
auc,0.827303,0.820124,0.848212,0.838457,0.837830,0.806527,0.822336,0.844873,0.773355,0.817250,0.862878,0.809930


In [187]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc']).mean(1)

acc    0.826927
f1     0.824892
pre    0.817100
rec    0.833418
auc    0.825756
dtype: float64

In [188]:
pd.DataFrame(pd.DataFrame(fi_dict).count(1).sort_values(ascending=False)[:10])

,0
returnsOpenPrevRaw5,12
pct_change,12
returnsClosePrevRaw1,9
returnsClosePrevMktres1,7
returnsClosePrevRaw7,7
returnsClosePrevMktres7,7
returnsClosePrevMktres3,6
Volume,6
returnsOpenPrevRaw7,5
his_vol_7,5


> ## 실험 4-2
- predictors : scl_li
- target : sh1_Y3_7

In [189]:
predictors = scl_li.copy()
target = 'sh1_Y3_7'
result, fi_dict = temp_calc_feature_importance(alg, train, test, predictors, target)

In [190]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc'])

,KB금융,LG화학,NAVER,POSCO,SK텔레콤,SK하이닉스,삼성전자,삼성중공업,셀트리온,현대건설,현대글로비스,현대차
acc,0.826446,0.822314,0.848739,0.838843,0.838843,0.809917,0.822314,0.847107,0.776860,0.818182,0.863636,0.809917
f1,0.820513,0.846975,0.820000,0.844622,0.828194,0.836879,0.821577,0.827907,0.800000,0.828125,0.845070,0.778846
pre,0.800000,0.862319,0.796117,0.828125,0.839286,0.848921,0.818182,0.831776,0.794118,0.803030,0.833333,0.750000
rec,0.842105,0.832168,0.845361,0.861789,0.817391,0.825175,0.825000,0.824074,0.805970,0.854839,0.857143,0.810000
auc,0.827303,0.820124,0.848212,0.838457,0.837830,0.806527,0.822336,0.844873,0.773355,0.817250,0.862878,0.809930


In [191]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc']).mean(1)

acc    0.826927
f1     0.824892
pre    0.817100
rec    0.833418
auc    0.825756
dtype: float64

In [192]:
pd.DataFrame(pd.DataFrame(fi_dict).count(1).sort_values(ascending=False)[:10])

,0
returnsOpenPrevRaw5,12
pct_change,12
returnsClosePrevRaw1,9
returnsClosePrevRaw7,7
returnsClosePrevMktres1,7
returnsClosePrevMktres7,7
scl_Volume,6
returnsClosePrevMktres3,6
returnsOpenPrevMktres3,5
his_vol_7,5


> ## 실험 4-3
- predictors : X_li
- target : sh5_Y3_7

In [193]:
predictors = X_li.copy()
target = 'sh5_Y3_7'
result, fi_dict = temp_calc_feature_importance(alg, train, test, predictors, target)

In [194]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc'])

,KB금융,LG화학,NAVER,POSCO,SK텔레콤,SK하이닉스,삼성전자,삼성중공업,셀트리온,현대건설,현대글로비스,현대차
acc,0.541322,0.561983,0.579832,0.611570,0.595041,0.657025,0.599174,0.553719,0.561983,0.632231,0.578512,0.500000
f1,0.602151,0.500000,0.596774,0.584071,0.577586,0.666667,0.597510,0.443299,0.475248,0.627615,0.604651,0.476190
pre,0.711864,0.398496,0.691589,0.528000,0.620370,0.601449,0.620690,0.401869,0.358209,0.576923,0.742857,0.509259
rec,0.521739,0.670886,0.524823,0.653465,0.540323,0.747748,0.576000,0.494253,0.705882,0.688073,0.509804,0.447154
auc,0.550993,0.590044,0.592308,0.617513,0.596432,0.663950,0.599966,0.540675,0.605815,0.637270,0.603217,0.500888


In [195]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc']).mean(1)

acc    0.581033
f1     0.562647
pre    0.563465
rec    0.590013
auc    0.591589
dtype: float64

In [196]:
pd.DataFrame(pd.DataFrame(fi_dict).count(1).sort_values(ascending=False)[:10])

,0
pct_change,11
his_vol_60,8
returnsOpenPrevRaw1,7
Volume,7
returnsClosePrevRaw3,6
his_vol_21,5
his_vol_91,5
returnsClosePrevMktres3,5
volume_ma_3,4
returnsOpenPrevRaw5,4


> ## 실험 4-4
- predictors : scl_li
- target : sh5_Y3_7

In [197]:
predictors = scl_li.copy()
target = 'sh5_Y3_7'
result, fi_dict = temp_calc_feature_importance(alg, train, test, predictors, target)

In [198]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc'])

,KB금융,LG화학,NAVER,POSCO,SK텔레콤,SK하이닉스,삼성전자,삼성중공업,셀트리온,현대건설,현대글로비스,현대차
acc,0.541322,0.561983,0.579832,0.611570,0.595041,0.657025,0.599174,0.553719,0.561983,0.632231,0.578512,0.500000
f1,0.602151,0.500000,0.596774,0.584071,0.577586,0.666667,0.597510,0.443299,0.475248,0.627615,0.604651,0.476190
pre,0.711864,0.398496,0.691589,0.528000,0.620370,0.601449,0.620690,0.401869,0.358209,0.576923,0.742857,0.509259
rec,0.521739,0.670886,0.524823,0.653465,0.540323,0.747748,0.576000,0.494253,0.705882,0.688073,0.509804,0.447154
auc,0.550993,0.590044,0.592308,0.617513,0.596432,0.663950,0.599966,0.540675,0.605815,0.637270,0.603217,0.500888


In [199]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc']).mean(1)

acc    0.581033
f1     0.562647
pre    0.563465
rec    0.590013
auc    0.591589
dtype: float64

In [200]:
pd.DataFrame(pd.DataFrame(fi_dict).count(1).sort_values(ascending=False)[:10])

,0
pct_change,11
his_vol_60,8
returnsOpenPrevRaw1,7
scl_Volume,7
returnsClosePrevRaw3,6
returnsClosePrevMktres3,5
his_vol_91,5
his_vol_21,5
returnsOpenPrevRaw5,4
scl_volume_ma_3,4


> ## 실험 5-1
- predictors : X_li
- target : sh1_Y3_10

In [201]:
predictors = X_li.copy()
target = 'sh1_Y3_10'
result, fi_dict = temp_calc_feature_importance(alg, train, test, predictors, target)

In [202]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc'])

,KB금융,LG화학,NAVER,POSCO,SK텔레콤,SK하이닉스,삼성전자,삼성중공업,셀트리온,현대건설,현대글로비스,현대차
acc,0.842975,0.830579,0.878151,0.826446,0.818182,0.842975,0.842975,0.855372,0.826446,0.822314,0.842975,0.772727
f1,0.845528,0.858131,0.851282,0.843284,0.796296,0.863309,0.851562,0.825871,0.852113,0.835249,0.828829,0.744186
pre,0.845528,0.849315,0.846939,0.849624,0.788991,0.909091,0.915966,0.813725,0.870504,0.851562,0.760331,0.808081
rec,0.845528,0.867133,0.855670,0.837037,0.803738,0.821918,0.795620,0.838384,0.834483,0.819549,0.910891,0.689655
auc,0.842932,0.822455,0.874644,0.825061,0.816684,0.848459,0.850191,0.852758,0.824458,0.822618,0.852609,0.769431


In [203]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc']).mean(1)

acc    0.833510
f1     0.832970
pre    0.842471
rec    0.826634
auc    0.833525
dtype: float64

In [204]:
pd.DataFrame(pd.DataFrame(fi_dict).count(1).sort_values(ascending=False)[:10])

,0
returnsOpenPrevRaw10,12
returnsOpenPrevRaw7,12
pct_change,12
returnsClosePrevRaw10,10
returnsOpenPrevMktres7,9
returnsClosePrevMktres10,5
returnsClosePrevRaw3,5
returnsClosePrevRaw7,5
returnsClosePrevRaw1,4
Volume,4


> ## 실험 5-2
- predictors : scl_li
- target : sh1_Y3_10

In [205]:
predictors = scl_li.copy()
target = 'sh1_Y3_10'
result, fi_dict = temp_calc_feature_importance(alg, train, test, predictors, target)

In [206]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc'])

,KB금융,LG화학,NAVER,POSCO,SK텔레콤,SK하이닉스,삼성전자,삼성중공업,셀트리온,현대건설,현대글로비스,현대차
acc,0.842975,0.830579,0.878151,0.826446,0.818182,0.842975,0.842975,0.855372,0.826446,0.822314,0.842975,0.772727
f1,0.845528,0.858131,0.851282,0.843284,0.796296,0.863309,0.851562,0.825871,0.852113,0.835249,0.828829,0.744186
pre,0.845528,0.849315,0.846939,0.849624,0.788991,0.909091,0.915966,0.813725,0.870504,0.851562,0.760331,0.808081
rec,0.845528,0.867133,0.855670,0.837037,0.803738,0.821918,0.795620,0.838384,0.834483,0.819549,0.910891,0.689655
auc,0.842932,0.822455,0.874644,0.825061,0.816684,0.848459,0.850191,0.852758,0.824458,0.822618,0.852609,0.769431


In [207]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc']).mean(1)

acc    0.833510
f1     0.832970
pre    0.842471
rec    0.826634
auc    0.833525
dtype: float64

In [208]:
pd.DataFrame(pd.DataFrame(fi_dict).count(1).sort_values(ascending=False)[:10])

,0
pct_change,12
returnsOpenPrevRaw10,12
returnsOpenPrevRaw7,12
returnsClosePrevRaw10,10
returnsOpenPrevMktres7,9
returnsClosePrevRaw3,5
returnsClosePrevRaw7,5
returnsClosePrevMktres10,5
returnsClosePrevRaw1,4
scl_Volume,4


> ## 실험 5-3
- predictors : X_li
- target : sh5_Y3_10

In [209]:
predictors = X_li.copy()
target = 'sh5_Y3_10'
result, fi_dict = temp_calc_feature_importance(alg, train, test, predictors, target)

In [210]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc'])

,KB금융,LG화학,NAVER,POSCO,SK텔레콤,SK하이닉스,삼성전자,삼성중공업,셀트리온,현대건설,현대글로비스,현대차
acc,0.768595,0.677686,0.710084,0.760331,0.710744,0.702479,0.586777,0.652893,0.466942,0.669421,0.801653,0.710744
f1,0.768595,0.680328,0.703863,0.756303,0.666667,0.727273,0.679487,0.614679,0.245614,0.666667,0.801653,0.660194
pre,0.762295,0.576389,0.803922,0.692308,0.666667,0.744186,0.929825,0.644231,0.155556,0.634921,0.815126,0.686869
rec,0.775000,0.830000,0.625954,0.833333,0.666667,0.711111,0.535354,0.587719,0.583333,0.701754,0.788618,0.635514
auc,0.768648,0.700211,0.719519,0.767413,0.705596,0.701350,0.676768,0.649328,0.514968,0.671190,0.801872,0.702942


In [211]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc']).mean(1)

acc    0.684862
f1     0.664277
pre    0.676024
rec    0.689530
auc    0.698317
dtype: float64

In [212]:
pd.DataFrame(pd.DataFrame(fi_dict).count(1).sort_values(ascending=False)[:10])

,0
returnsClosePrevRaw5,12
pct_change,10
his_vol_7,5
volume_ma_25,5
returnsOpenPrevRaw5,5
his_vol_30,4
volume_ma_5,4
returnsClosePrevMktres1,4
Open,4
volume_ma_23,4


> ## 실험 5-4
- predictors : scl_li
- target : sh5_Y3_10

In [213]:
predictors = scl_li.copy()
target = 'sh5_Y3_10'
result, fi_dict = temp_calc_feature_importance(alg, train, test, predictors, target)

In [214]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc'])

,KB금융,LG화학,NAVER,POSCO,SK텔레콤,SK하이닉스,삼성전자,삼성중공업,셀트리온,현대건설,현대글로비스,현대차
acc,0.768595,0.677686,0.710084,0.760331,0.710744,0.702479,0.586777,0.652893,0.466942,0.669421,0.801653,0.710744
f1,0.768595,0.680328,0.703863,0.756303,0.666667,0.727273,0.679487,0.614679,0.245614,0.666667,0.801653,0.660194
pre,0.762295,0.576389,0.803922,0.692308,0.666667,0.744186,0.929825,0.644231,0.155556,0.634921,0.815126,0.686869
rec,0.775000,0.830000,0.625954,0.833333,0.666667,0.711111,0.535354,0.587719,0.583333,0.701754,0.788618,0.635514
auc,0.768648,0.700211,0.719519,0.767413,0.705596,0.701350,0.676768,0.649328,0.514968,0.671190,0.801872,0.702942


In [215]:
pd.DataFrame(result, index=['acc','f1','pre','rec','auc']).mean(1)

acc    0.684862
f1     0.664277
pre    0.676024
rec    0.689530
auc    0.698317
dtype: float64

In [216]:
pd.DataFrame(pd.DataFrame(fi_dict).count(1).sort_values(ascending=False)[:10])

,0
returnsClosePrevRaw5,12
pct_change,10
scl_volume_ma_25,5
his_vol_7,5
returnsOpenPrevRaw5,5
scl_volume_ma_5,4
scl_Open,4
scl_volume_ma_23,4
his_vol_30,4
scl_close_ma_29,4
